In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from path import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
#Mockup data that would be gathered from database already cleaned

file_path = Path("data/cleaned_Housing_data.csv")

df = pd.read_csv(file_path)

df.head()

In [ ]:
#Target Variable is Price. Thats what we want to predict
#We want to see the relationship between Year and price
plt.scatter(df.Years, df.price)
plt.xlabel("Years ")
plt.ylabel("Housing price USD $")
plt.show()

In [ ]:
#Declaring features and targets

X = df.drop("Target_price")
y=df.Target_price

In [ ]:
#instantized and fit data to model and predictions
model = LinearRegression()

model.fit(X, y)

y_pred = model.predict(X)

In [ ]:
#Print the coeffiecient and intercept of the nodel

print(model.coef_)
print(model.intercept_)

In [ ]:
#Print regression line

plt.scatter(X,y)
plt.plot(X, y_pred, color="red")
plt.show()